A continuación, se describen las actividades a realizar:
1. Representar gráficamente cada una de las funciones para el caso n=2. Aunque, como se verá más adelante, el problema de encontrar el mínimo se planteará, en ambas funciones, para valores de n>2, la visualización 2D permitirá al alumno hacerse una idea de la complejidad del problema a resolver.
2. Implementar el código de una estrategia evolutiva y describir las características de la aplicación implementada. Al entregar la práctica, el alumno deberá adjuntar el código fuente y el ejecutable.
3. Aplicar el programa implementado para encontrar el mínimo de la función hiper-elipsoide rotado. Para ello, se usará la siguiente configuración: mutación no correlacionada de paso único, número de variables n=10, estrategia de selección de supervivientes de tipo (μ,λ)=(30, 200). Realizar 20 ejecuciones independientes de 1000 generaciones cada una.
4. Repetir el experimento anterior utilizando mutación no correlacionada de n pasos.
5. Repetir el paso (3) con la estrategia (μ+λ).
6. Repetir el paso (4) con la estrategia (μ+λ).
7. Para facilitar la comparativa de los resultados obtenidos en los pasos (3-6), se debe representar en una tabla los valores SR, MBF y AES (ver capítulo 14). En virtud de estos resultados aquí obtenidos, justifique cuál es la configuración más idónea para resolver el problema planteado.
8. Dado que una estrategia evolutiva depende de otros parámetros, tales como el tipo de recombinación, tamaño de la población, valores de épsilon mínimo, tau y tau’, ¿es posible mejorar el comportamiento del algoritmo actuando sobre algunos de estos parámetros?
9. Repetir los pasos (3-8) para la función de Rastrigin.

#1. Representación gráfica de las funciones para dos dimensiones

Primero vamos a definir una función que nos permite dibujar en 3 dimensiones. Utilizamos la librería [matplotlib](http://matplotlib.org/), bastante conocida en el mundo [python](https://www.python.org/).

In [154]:
%matplotlib inline

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import random
import math 
from matplotlib import cm


def plot_3d(f,a,b,step,figsize=(15, 15)):
    """
    Dibuja en tres dimensiones una función 
    de dos variables

    Args:
        f (function): la función
        a (float): extremo izquierdo del 
            dominio para las dos variables de f
        b (float): extremo derecho del dominio 
            para  las dos variables de f
        step (float): granularidad con la 
            que se  muestrea f
        figsize (int,int): tamaño de la figura
    """
    x = y = np.arange(a,b,step)
    X, Y = np.meshgrid(x, y)
    z = np.array([f([x,y]) for x,y in zip(np.ravel(X), 
                                          np.ravel(Y))])
    Z = z.reshape(X.shape)
        
    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.plot_surface(X, Y, Z, cmap=cm.coolwarm_r)   

Defino ahora la función hiper-elipsoide rotado y la función de Rastrgin.

In [155]:
def hiper_elipsoide_rotado(coords):
    """
    Devuelve el valor del hiper-elipsoide 
    rotado en el punto dado por coords.

    Args:
        coords: las coordenadas de un 
            punto en un espacio real de 
            dimensión n
    """
    result = 0
    for i in range(1, len(coords)+1):
        for j in range(1, i+1):
            result += coords[j-1]**2
    return result

def rastrigin(coords):
    """
    Devuelve el valor de la función de 
    Rastrigin en el punto dado por coords.

    Args:
        coords: las coordenadas de un punto 
            en un espacio real de dimensión n
    """
    result = 0
    for xi in coords:
        result += (xi**2 - 10*math.cos(2*math.pi*xi))
    return 10*len(coords) + result

Podemos dibujar ahora el hiper-elipsoide rotado de 2 dimensiones:

In [22]:
# plot_3d(hiper_elipsoide_rotado, -65.54 , 65.54 , 1)

Y la función de Rastrigin:

In [10]:
# plot_3d(rastrigin, -5.12 , 5.12 , 0.02)

En el caso de la función de Rastrigin, claramente vemos que el renderizado en 3 dimensiones no nos dá suficiente información visual (tiene tantos máximos y mínimos que acaba uno por no saber como es la función).

Por eso, será conveniente disponer de la siguiente función que nos permite tener una vista cenital de las funciones representando el valor que adoptan las mismas como un mapa de calor.

In [156]:
def plot_color_map(f,a,b,step,figsize=(15, 9)):
    """
    Dibuja en dos dimensiones la vista 
    cenital de una función de dos variables 
    vista como un mapa de intensidad.

    Args:
        f: la función
        a: extremo izquierdo del dominio 
            para las dos variables de f
        b: extremo derecho del dominio 
            para las dos variables de f
        step: granularidad con la que se
            muestrea f
        figsize: tamaño de la figura
    """
    x = y = np.arange(a,b,step)
    X, Y = np.meshgrid(x, y)
    z = np.array([f([x,y]) for x,y 
                  in zip(np.ravel(X), np.ravel(Y))])
    Z = z.reshape(X.shape)

    fig = plt.figure(figsize=(20, 13))
    ax = fig.add_subplot(111)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.imshow(Z,extent=[a,b,a,b])
    plt.colorbar(orientation='vertical')

Con esta función podemos ver que efectivamente el hiper elipsoide rotado se corresponde con una superficie con un único mínimo en el origen:

In [94]:
# plot_color_map(hiper_elipsoide_rotado, -65.54 , 65.54 , 1)

Del mismo modo, podemos tener una vista más intuitiva del aspecto de la función de Rastrigin para 2 dimensiones. Observamos que en el intervalo en el que se dibuja tiene muchos mínimos locales, cuyo valor se hace más pequeño a medida que se aproximan al origen de coordenadas donde se encuentra su mínimo global.

In [66]:
# plot_color_map(rastrigin, -5.12 , 5.12 , 0.02)

# Implementación de la estrategia evolutiva de búsqueda de mínimos globales

Siguiendo las indicaciones de la práctica, para la búsqueda de los mínimos globales de las funciones hiper elipsoide rotado y función de Rastrigin, se ha inplementado una estrategia evolutiva.

Este tipo de estratedias están principalmente indicadas en problemas de optimización numérica (como es el caso de los problemas que queremos resolver). Son rápidas y suelen funcionar bien en problemas en los que se pueden representar las soluciones con números reales.

A lo largo de la evolución, en este tipo de estrategias, no sólo va evolucionando la solución sino también los parámetros que controlan la evolución de la misma. 

## Introducción

Mi implemetación es lo suficientemente genérica como para resolver problemas de cualquier dimensión n.

En el caso de mi implementación, represento los individuos como diccionarios con las propiedades:
 - coords (coordenadas del individuo) 
 - sigmas (parámetros sigma para el operador de mutación).

Dado que en la práctica no se piden mutaciones correlacionadas, no agrego en la representación los parámetros alfa.

Además, dado que en la práctica se piden experimentos de paso único y de n pasos determino que la propiedad sigmas de un individuo puede:
 - tener un único elemento numérico, en cuyo caso el comportamiento del algoritmo será de paso único
 - o bien tener un array con el mismo número de elementos que coords, en cuyo caso el comportamiento del algoritmo será de n pasos

Será responsabilidad de la implementación de mis algoritmos interpretar, según la representación de los individuos, si la computación debe realizarse desde el enfoque de paso único (misma sigma controla la variabilidad de todas las coordenadas) o de n pasos (un sigma independiente para el control de la variabilidad de cada coordenada).

Por ejemplo, para el caso de 3 dimensiones, un ejemplo de individuo perteneciente a una ejecución configurada con paso único sería:

```python
individual_1 = {'coords':[1,2,3],'sigmas':[1]}
```

Y para el mismo caso de 3 dimensiones, un ejemplo de individuo perteneciente a una ejecución configurada con n pasos sería:

```python
individual_2 = {'coords':[1,2,3],'sigmas':[2,3,1]}
```

## Implementación

In [163]:
from collections import Iterable 
import math

def rand_individual(domain,dimension,step='n',sigma=1):    
    """
    Genera un inviduo aleatoriamente.
    Para conocer la representación utilizada para
    los individuos, léanse los explicaciones 
    anteriores.

    Args:
        domain: intervalo real de definición de la 
            función cuyos óptimos buscamos
        dimension: dimensión del experimento
        step: 'n' si el individuo va a participar 
            en una estrategia de n pasos o 'one' 
            si la estrategia es de paso único
        sigma: valor que adoptará/n el/los sigma/s 
            del individuo dependiendo de que la 
            estrategia sea de paso único o de n pasos
    """
    coords=np.random.uniform(domain[0],domain[1],dimension)
    if step=='one':
        sigmas=[sigma]
    elif step=='n':
        sigmas=[sigma]*dimension
    else:
        raise ValueError('Not a valid step')
    return {'coords':coords,'sigmas':sigmas}


def one_step_mutation(individual,min_sigma=0):
    """
    Muta un individuo en experimentos 
    de paso único.

    Args:
        individual: el individuo a mutar
        min_sigma: el valor mínimo de sigma 
            permitido en el individuo mutado
    """
    coords,[sigma]=individual['coords'],individual['sigmas']
    n=len(coords)
    tau=1/math.sqrt(n)
    sigma_mut=sigma*math.exp(tau*np.random.normal())
    sigma_mut=sigma_mut if min_sigma<sigma_mut else sigma
    coords_mut=[x+sigma_mut*np.random.normal() for x in coords]
    return {'coords':coords_mut,'sigmas':[sigma_mut]}


def n_step_mutation(individual,min_sigma=0):
    """
    Muta un individuo en experimentos de n pasos.

    Args:
        individual: el individuo a mutar
        min_sigma: el valor mínimo de sigma 
            permitido en los sigmas del individuo mutado
    """
    coords,sigmas=individual['coords'],individual['sigmas']
    n=len(coords)
    tau_1=1/math.sqrt(2*n)
    tau_2=1/math.sqrt(2*math.sqrt(n))
    common_norm=np.random.normal()
    sigmas_mut=[]
    coords_mut=[]
    for sigma,coord in zip(sigmas,coords):
        sigma_mut=sigma*math.exp(tau_1*common_norm + 
                                 tau_2*np.random.normal())
        sigma_mut=sigma_mut if min_sigma<sigma_mut else sigma
        coord_mut=coord + sigma_mut*np.random.normal()
        sigmas_mut += [sigma_mut]
        coords_mut += [coord_mut]
    return {'coords':coords_mut,'sigmas':sigmas_mut}


def discrete_recombination(ind_1,ind_2):
    """
    Recombina dos individuos por el método 
    discreto (tomando una a una la coordenada 
    y el sigma de uno u otro individuo, 
    según el azar).
    Este método acepta tanto individuos que 
    participan en estrategias de paso único 
    como de n pasos.

    Args:
        ind_1: primer individuo
        ind_2: segundo individuo
        
    Returns:
        la recombinación de ambos individuos
    """
    sigmas_comb=[]
    coords_comb=[]
    both=[ind_1,ind_2]
    one_step= 1==len(ind_1['sigmas'])
    for i in range(len(ind_1['coords'])):
        j=np.random.choice([0,1])
        coords_comb+=[both[j]['coords'][i]]
        if one_step:
            sigmas_comb=[both[j]['sigmas'][i]]
        else:
            sigmas_comb+=[both[j]['sigmas'][i]]
    return {'coords':coords_comb,'sigmas':sigmas_comb}


def intermediate_recombination(ind_1,ind_2):
    """
    Recombina dos individuos por el método 
    intermedio (tomando una a una la 
    coordenada y el sigma mediosde ambos 
    individuos).
    Este método acepta tanto individuos que 
    participan en estrategias de paso único 
    como de n pasos.

    Args:
        ind_1: primer individuo
        ind_2: segundo individuo
    """
    return {'coords':[(a+b)/2 for a,b in zip(ind_1['coords'],ind_2['coords'])],
            'sigmas':[(a+b)/2 for a,b in zip(ind_1['sigmas'],ind_2['sigmas'])]}



def mu_comma_lambda(mus,lambdas,f,mu):
    """
    Selección de tipo mu comma lambda
    (selecciona los mejores individuos del 
    conjunto de lambas/hijos).

    Args:
        mus: los padres de la generación 
            en curso
        lambdas: los hijos de la generación 
            en curso 
        f: la función de evaluación de la 
            calidad de los individuos 
            (función fitness)
        mu: número de individuos a seleccionar
    """
    return sorted(lambdas, key=lambda x:f(x['coords']))[0:mu]


def mu_plus_lambda(mus,lambdas,f,mu):
    """
    Selección de tipo mu plus lambda
    (selecciona los mejores individuos del 
    conjunto de mus/padres y lambas/hijos).

    Args:
        mus: los padres de la generación en curso
        lambdas: los hijos de la generación en curso 
        f: la función de evaluación de la calidad
            de los individuos (función fitness)
        mu: número de individuos a seleccionar
    """
    return sorted(mus+lambdas, 
                  key=lambda x:f(x['coords']))[0:mu]


def termination_cond_met(best,generation,max_generations,
                         termination_delta,f):
    """
    Determina si se dan las condiciones para que 
    la evolución acabe por
     - haber evolucionado durante demasiadas 
         generaciones
     - haber alcanzado el mejor individuo de la 
         generación en curso un fitness 
         suficientemente bueno

    Args:
        best: array con el histórico de los 
            mejores individuos de cada generación
        generation: generación en curso
        max_generations: máximo número de 
            generaciones permitido en el experimento
        termination_delta: error mínimo del mejor 
            individuo. Si el fitness del mejor 
            individuo (cercanía al mínimo global 
            que sabemos que es 0) es inferior 
            a este valor, el experimento se da 
            por concluido.
        f: la función de evaluación de la calidad
            de los individuos (función fitness)
    """    
    return max_generations<=generation \
           or 0<len(best) \
              and abs(f(best[-1]['coords']))<termination_delta


def live(domain, dimension, f, 
         step='n', selection='mu_comma_lambda', 
         recombination='intermediate',
         mu=30, lamb=200,
         sigma=1, min_sigma=0.0001, 
         max_generations=1000,
         termination_delta=0.0001):
    """
    Lleva a cabo la estrategia evolutiva de 
    búsqueda de mínimos globales de las funciones 
    hiper elipsoide rotado o función de Rastrigin, 
    tal como se ha descrito  en el presente 
    documento.

    Args:
        domain: dominio de definición de la 
            función cuyos mínimos globales 
            se buscan
        dimension: demensión del espacio en el 
            que se define la función f
        f: función cuyos mínimos globales 
            buscamos con la estrategia evolutiva
        step: determina si la estrategia será 
            de paso único 'one' o de n pasos 'n'. 
            El valor por defecto es 'n'.
        selection: determina si la selección 
            de hijos  se hará con el método 
            mu comma lambda 'mu_comma_lambda' 
            o mu plus lambda  'mu_plus_lambda'.
            Valor por defecto 'mu_comma_lambda'.
        recombination: determina si la recombinación 
            de individuos se hará por el método 
            discreto 'discrete' o intermedio
            'intermediate'.
            Valor por defecto 'intermediate'
        mu: número de padres en cada iteración. 
            El valor por defecto es de 30.
        lamb: número de hijos en cada interación.
            El valor por defecto es de 200.
        sigma: valor por defecto que se aplicará 
            en la  inicialización de los sigmas 
            de todo individuo.
            Valor por defecto 1.
        min_sigma: valor mínimo permitido de los 
            sigmas de un individuo.
            Valor por defecto 0.0001.
        max_generations: número de generaciones 
            máximo permitido en el experimento. 
            Valor por defecto 1000:
        termination_delta: error mínimo del mejor 
            individuo. 
            Si el fitness del mejor individuo 
            (cercanía al mínimo global que sabemos
            que es 0) es inferior a este valor, 
            el experimento se da por concluido.
            Valor por defecto 0.0001.      

    Returns:
        - el mejor individuo encontrado tras la evolución 
        - y un array con el histórico del mejor inviduo 
            en cada generación.
    """    
    
    if step=='n':
        mut_f=n_step_mutation
    elif step=='one': 
        mut_f=one_step_mutation
    
    if recombination=='intermediate':
        rec_f=intermediate_recombination
    elif recombination=='discrete':
        rec_f=discrete_recombination
        
    if selection=='mu_comma_lambda':
        sel_f=mu_comma_lambda
    elif selection=='mu_plus_lambda':
        sel_f=mu_plus_lambda
    
    t,best=0,[]
    mus=[rand_individual(domain,dimension,step,sigma) for i in range(mu)]    
    while not termination_cond_met(best,t,max_generations,termination_delta,f):
        recombinations=[rec_f(*random.sample(mus,2)) for i in range(lamb)]
        mutations=[mut_f(ind,min_sigma) for ind in recombinations]
        mus=sel_f(mus,mutations,f,mu)
        best+=[mus[0]]
        t+=1
    return best[-1],best

##Descripción de la estrategia

Observando la función live, que es la que coordina la evolución, se pueden apreciar los aspectos que siguen.

Dado un experimento, es obligatorio indicar el número de dimensiones que se van a utilizar, la función f (hiperelipsoide rotada o rastrigin) y el dominio de definición de la función f.

Todo experimento puede realizarse de paso único o de n pasos, con recombinación intermedia o discreta y con selección de descendencia por $(\mu,\lambda)$ o $(\mu + \lambda)$.

Los valores de mu, lamb, sigma, min_sigma, max_generations o termination_delta se dan por defecto, aunque pueden ser configurados en cada ejecución del experimento.

La estrategia en sí es bastante simple, un individuo se representa por unas coordenadas en el espacion n-dimensional y por uno ('one' step) o tantos sigmas como coordenadas tenga el individuo ('n' step), que gobierna la intensidad de la mutación tanto de las coordenadas como de los propios sigmas.

En esta implementación, por hacerla más sencilla de entender y de mantener, se toman los valores de tau por defecto que se indican en el documento base de referencia. No se permite parametrizar tales valores sino que todos los experimentos tendrán los mismos en función del número de dimensiones.

Con estas explicaciones, el algoritmo simplemente hace esto:
 - genera una población inicial al azar en el intervalo de definición de la función
 - mientras el fitness del mejor individuo no sea suficientemente bueno o no se haya alcanzado el máximo de generaciones permitido
   - generar lamd individuos recombinando la población actual
   - mutar los individuos lamb anteriores
   - establecer la nueva población como los mejores individuos seleccionados de entre los individuos mutados y la población actual (dependiendo de la función de selección)
   - guardar el mejor de los resultantes en el paso anterior en el histórico de mejores individuos por generación
   - repetir hasta que no se tenga un individuo con suficiente fitness o no se hayan agotado el máximo de generaciones permitidas
 - devolvel el mejor individuo y el histórico de mejores individuos por generación
 
##Pruebas en dos dimensiones 

Antes de pasar a hacer los experimentos en 10 dimensiones que se piden en la práctica es mejor ver como se comporta el algoritmo en el caso de 2 dimensiones.

La ventaja de hacerlo en 2 dimensiones es que podemos disponer de un gráfico qeu muestre visualmente "la traza" de la evolución.

A tal efecto, la siguiente función nos será muy útil junto a la función plot_color_map que se definión anteriormente.

In [164]:
def add_scatter(individuals):
    """
    Dibuja un scatter con la secuencia de 
    coordenadas que se corresponden con una 
    secuencia de individuos según los 
    represento en la presente práctica
    (se explica más arriba).
    Permite representar el camino que sigue la 
    evolución de la mejor solución que el 
    algoritmo encuentra en cada generación.

    Args:
        individuals: array de individuos
    """    
    x,y=[],[]
    for ind in individuals:
        x+=[ind['coords'][0]]
        y+=[ind['coords'][1]]
    colors=cm.rainbow(np.linspace(0, 1, len(x)))
    plt.scatter(x, y, c=colors, s=100)

Igualmente, nos será uy útil una función que dibuje como de próximo está un individuo al óptimo global buscado, que sabemos que en ambos casos es 0. Algo que nos sirva para representar la evolución del fitness del mejor individuo.

**Nótese que la siguiente función hace una transformación logarítmica de los valores por f de cada individuo, para evitar que los gráficos salgan aberrados y por lo tanto que no sirvan para interpretarse con provecho.**

In [120]:
def plot_inv_log_fitness(f,individuals):
    """
    Dibuja el valor dado por f en una población 
    de individuos en escala logarítmica. 
    Puede interpretarse con el inverso
    del fitness de los individuos.
    
    Se utiliza escala logarítmica porque si no
    los gráficos salen un poco aberrados.

    Args:
        f: una función n variada
        individuals: array de individuos con 
            n coordenadas cada uno.
    """    
    plt.plot([math.log(f(ind['coords'])+1) for ind in individuals])

###Hiperelipsoide rotado en 2 dimensiones

De este modo, para el hiper elipsoide rotado de 2 dimensiones, con todos los valores por defecto, podemos llevar a cabo una búsqueda del mínimo global:

In [110]:
best,all=live([-65.54 , 65.54],2,hiper_elipsoide_rotado)

Y ver gráficamente como el mejor individuo de cada generación se va aproximando al origen de coordenadas.
Nótese que el aprendizaje es tan rápido que los individuos muy rápidamente se ubican muy próximos al origen de coordenadas. Por eso, reduzco a [-5,5] el intervalo en el que dibujo los gráficos.

In [114]:
# plot_color_map(hiper_elipsoide_rotado,-5,5,0.1)
# add_scatter(all)

Podemos ver la evolución del fitness del mejor individuo (en escala logarítmica e inversa):

In [116]:
# plot_inv_log_fitness(hiper_elipsoide_rotado,all)

###Rastrigin en 2 dimensiones

Equivalenteme, para el caso de la función de Rastrigin, en dos dimensiones, tenemos:

In [136]:
best,all=live([-5.12 , 5.12],2,rastrigin)

# Reducimos el intervalo en el que dibujamos
# para mayor claridad del dibujo
# plot_color_map(rastrigin,-2,2,0.1)
# add_scatter(all)
# plt.show()
# plot_inv_log_fitness(rastrigin,all)

Nótese que si en este ejemplo no se indica un valor mínimo para los sigmas, normalmente la solución se estanca en un mínimo local y no llega al global. Sin embargo, si limitamos por abajo el valor de los sigmas (como ocurre por defecto), en mis experimentos siempre he llegado al delta de error mínimo establecido.

Y son esto, más o menos, podemos hacernos una idea del herramentaje que tenemos a nuestra disposición para continuar con la práctica.

#Ejercicio 3: Búsqueda del mínimo del hiper-elipsoide rotado de 10 dimensiones con selección mu coma lambda y paso único

Realizamos 20 ejecuciones y guardamos los resultados en la variable **hiper_10_30_200_one_comma_results**.

Analizaremos estos resultados posteriormente.

In [139]:
def hiper_10_30_200_one_comma():
    return live([-65.54 , 65.54],10,
                hiper_elipsoide_rotado,
                step='one',  
                selection='mu_comma_lambda',
                mu=30,lamb=200)

# hiper_10_30_200_one_comma_results = [hiper_10_30_200_one_comma()
#                                     for i in range(20)]

#Ejercicio 4: Búsqueda del mínimo del hiper-elipsoide rotado de 10 dimensiones con selección mu coma lambda y paso n

Realizamos 20 ejecuciones y guardamos los resultados en la variable **hiper_10_30_200_n_comma_results**.

Analizaremos estos resultados posteriormente.

In [143]:
def hiper_10_30_200_n_comma():
    return live([-65.54 , 65.54],10,
                hiper_elipsoide_rotado,
                step='n',  
                selection='mu_comma_lambda',
                mu=30,lamb=200)

hiper_10_30_200_n_comma_results = [hiper_10_30_200_n_comma()
                                   for i in range(20)]

#Ejercicio 5: Búsqueda del mínimo del hiper-elipsoide rotado de 10 dimensiones con selección mu plus lambda y paso único

Realizamos 20 ejecuciones y guardamos los resultados en la variable **hiper_10_30_200_one_plus_results**.

Analizaremos estos resultados posteriormente.

In [145]:
def hiper_10_30_200_one_plus():
    return live([-65.54 , 65.54],10,
                hiper_elipsoide_rotado,
                step='one',  
                selection='mu_plus_lambda',
                mu=30,lamb=200)

# hiper_10_30_200_one_plus_results = [hiper_10_30_200_one_plus()
#                                     for i in range(20)]

#Ejercicio 6: Búsqueda del mínimo del hiper-elipsoide rotado de 10 dimensiones con selección mu plus lambda y paso n

Realizamos 20 ejecuciones y guardamos los resultados en la variable hiper_10_30_200_n_plus_results.

Analizaremos estos resultados posteriormente.

In [146]:
def hiper_10_30_200_n_plus():
    return live([-65.54 , 65.54],10,
                hiper_elipsoide_rotado,
                step='n',  
                selection='mu_plus_lambda',
                mu=30,lamb=200)

# hiper_10_30_200_n_plus_results = [hiper_10_30_200_n_plus()
#                                   for i in range(20)]

#Resultados para la función hiper-elipsoide rotado

Para justificar la conveniencia de una otra configuración de la estrategia evolutiva utilizada para la búsqueda de los mínimos del hiper-elipsoide rotado, primero tenemos que definir las funciones de rendimiento con las que medir los resultados.

Lógicamente, estas funciones serán igualmente utilizables en el caso de la función de Rastrigin.

In [153]:
def AES(results):
    """
    Calcula el número medio de evaluaciones 
    hasta dar con la solución.

    Args:
        results (array): un array con la salida 
            devuelta por la función live, i.e. un 
            array  de arrays con mejor individuo
            encontrado como primer elemento
            e histórico de mejores individuos 
            por generación como segundo elemento.
            
    Returns:
        float: el número medio de evaluaciones 
            hasta dar con la solución
    """
    return None

AES(None)